# 🤖 Ders 2 Uygulaması: Denetimsiz Öğrenme ve Dinamik Müşteri Segmentasyonu

Makine öğrenmesi projelerinde her zaman elimizde etiketlenmiş, hazır bir "cevap anahtarı" bulunmaz. Müşteri kitlelerini analiz etmek, finansal anomalileri tespit etmek veya pazar araştırması yapmak istediğimizde, algoritmaların veriler içindeki gizli örüntüleri kendi kendine bulması gerekir. Makinenin veriyi insan yönlendirmesi olmadan bağımsız bir şekilde organize ettiği bu felsefeye **Gözetimsiz Öğrenme (Unsupervised Learning)** denir.

### K-Means Algoritması ile Kümeleme (Clustering)
Bu uygulamada kullanılan K-Means algoritması, veriyi matematiksel yakınlıklarına (Öklid uzaklığı) göre önceden belirlenmiş sayıda (K) kümeye ayırır:
1. **Merkez Belirleme:** Uzayda rastgele merkez noktaları (Centroid) seçilir.
2. **Atama:** Her bir veri noktası (müşteri), kendisine matematiksel olarak en yakın olan merkeze atanır.
3. **Optimizasyon:** Oluşan yeni kümelerin ağırlık merkezi tekrar hesaplanır ve merkez noktaları o yöne kaydırılır. Merkezler sabitlenene kadar bu optimizasyon devam eder.

### Karar Destek Sistemi (DSS) Olarak Makine Öğrenmesi
Aşağıdaki kod bloğu yalnızca statik bir görselleştirme aracı değildir; rastgele üretilen, boyutları sürekli değişen veri setleri üzerinde çalışan dinamik bir Karar Destek Sistemidir (Decision Support System). 

Kodu her çalıştırdığınızda pazar demografisi tamamen değişecektir. Sistem çalışma anında (runtime) şu mühendislik adımlarını uygular:
* `bincount` fonksiyonu ile kümelerin içine düşen müşteri sayılarını hesaplar.
* `argmax` fonksiyonu ile pazarın en büyük dilimini (hedef kitleyi) otomatik olarak tespit eder.
* O anki baskın müşteri profiline (Yıldız, Savurgan, Tedbirli vb.) en uygun **Pazarlama ve Satış Stratejisini** veriye dayalı olarak üretir.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings

warnings.filterwarnings('ignore')

print("DERS 2 UYGULAMASI: GÖZETİMSİZ ÖĞRENME - DİNAMİK MÜŞTERİ SEGMENTASYONU\n")

# --- 1. ADIM: TAMAMEN RASTGELE HAM VERİ ÜRETİMİ ---
# Her grupta 30 ile 150 arasında rastgele sayıda müşteri olacak.
grup_sayilari = np.random.randint(30, 150, size=5)

c1 = np.random.normal(loc=[60, 50], scale=[10, 10], size=(grup_sayilari[0], 2))
c2 = np.random.normal(loc=[100, 20], scale=[12, 8], size=(grup_sayilari[1], 2))
c3 = np.random.normal(loc=[100, 80], scale=[12, 8], size=(grup_sayilari[2], 2))
c4 = np.random.normal(loc=[25, 80], scale=[8, 8], size=(grup_sayilari[3], 2))
c5 = np.random.normal(loc=[25, 20], scale=[8, 8], size=(grup_sayilari[4], 2))

ham_veri = np.vstack((c1, c2, c3, c4, c5))
ham_veri[:, 1] = np.clip(ham_veri[:, 1], 1, 100)
toplam_musteri = len(ham_veri)

print(f"BİLGİ: Toplam {toplam_musteri} müşterinin rastgele üretilen Gelir ve Harcama verisi sisteme yüklendi.\n")
print("DİKKAT: Makineye hiçbir müşteri profili veya etiketi (Label) verilmedi.\n")

# --- 2. ADIM: K-MEANS ALGORİTMASININ KURULUMU ---
k_means = KMeans(n_clusters=5, random_state=None, n_init=10)
k_means.fit(ham_veri)

kumeler = k_means.labels_
merkezler = k_means.cluster_centers_

# --- 3. ADIM: GÖRSELLEŞTİRME ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
fig.canvas.manager.set_window_title("Gözetimsiz Öğrenme: K-Means")

ax1.scatter(ham_veri[:, 0], ham_veri[:, 1], c='gray', s=30, alpha=0.6)
ax1.set_title("Makineden Önce (Etiketsiz Ham Veri)", fontweight='bold', pad=15)
ax1.set_xlabel("Yıllık Gelir (Bin TL)")
ax1.set_ylabel("Harcama Puanı (1-100)")
ax1.grid(True, alpha=0.3)

ax2.scatter(ham_veri[:, 0], ham_veri[:, 1], c=kumeler, cmap='tab10', s=40, alpha=0.8)
ax2.scatter(merkezler[:, 0], merkezler[:, 1], c='red', s=250, marker='X', edgecolors='black', label='Merkez')
ax2.set_title("Makineden Sonra (K-Means Segmentasyonu)", fontweight='bold', pad=15)
ax2.set_xlabel("Yıllık Gelir (Bin TL)")
ax2.set_ylabel("Harcama Puanı (1-100)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- 4. ADIM: DİNAMİK VE AKILLI MÜHENDİSLİK RAPORU ---
print("DİNAMİK MÜŞTERİ SEGMENTASYONU VE İŞ KARARI RAPORU\n")

kume_boyutlari = np.bincount(kumeler)
bulunan_profiller = []

for i in range(5):
    gelir = merkezler[i, 0]
    harcama = merkezler[i, 1]
    kisi_sayisi = kume_boyutlari[i]
    yuzde = (kisi_sayisi / toplam_musteri) * 100
    
    if gelir >= 75 and harcama >= 60:
        profil = "YILDIZ KİTLE"
    elif gelir >= 75 and harcama < 60:
        profil = "TEDBİRLİ"
    elif gelir < 45 and harcama >= 60:
        profil = "SAVURGAN"
    elif gelir < 45 and harcama < 60:
        profil = "DÜŞÜK PROFİL"
    else:
        profil = "STANDART"
        
    bulunan_profiller.append(profil)
    print(f"Küme {i+1} [{profil:12}]: %{yuzde:4.1f} pazar payı ({kisi_sayisi} kişi) -> Ortalama Gelir: {gelir:4.1f}k, Harcama: {harcama:4.1f}")

en_buyuk_kume_idx = np.argmax(kume_boyutlari)
hedef_kitle = bulunan_profiller[en_buyuk_kume_idx]
hedef_yuzde = (kume_boyutlari[en_buyuk_kume_idx] / toplam_musteri) * 100

print("\nMAKİNE ÖĞRENMESİ ÇIKARIMI VE STRATEJİ ÖNERİSİ:")
print(f"Sistem, %{hedef_yuzde:.1f} oranla pazarın en büyük dilimini '{hedef_kitle}' segmentinin oluşturduğunu tespit etmiştir.")

if "YILDIZ KİTLE" in hedef_kitle:
    print("-> STRATEJİ: Şirketin ana geliri sadık ve yüksek harcama yapan müşterilerden gelmektedir. Bu grubu elde tutmak için VIP sadakat programları başlatılmalıdır.")
elif "TEDBİRLİ" in hedef_kitle:
    print("-> STRATEJİ: Müşteri tabanının yüksek geliri var ancak harcama yapmaktan kaçınıyorlar. İndirim kampanyaları ve promosyonlarla harcamaya teşvik edilmeliler.")
elif "SAVURGAN" in hedef_kitle:
    print("-> STRATEJİ: Çoğunluk düşük gelirli olmasına rağmen yüksek harcama eğiliminde. Satışları artırmak için kredi kartına ek taksit ve esnek ödeme seçenekleri sunulmalıdır.")
elif "STANDART" in hedef_kitle:
    print("-> STRATEJİ: Pazarın geneli ortalama gelir ve harcamaya sahip. Marka bilinirliğini artırarak bu grubu üst segmentlere taşımaya yönelik reklam bütçesi ayrılmalıdır.")
else:
    print("-> STRATEJİ: Pazarın çoğunluğu düşük gelirli ve harcama yapmayan profilde. Yeni müşteri kazanımına odaklanılmalı ve daha uygun fiyatlı alt ürün segmentleri oluşturulmalıdır.")